In [1]:
using Compat
using QPControl
using RigidBodyDynamics
using AtlasRobot
using HumanoidLCMSim
using HumanoidLCMSim.AtlasSim
using Compat.Test

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9ed50fe60e4c3cfba5d8d51afc0a1ba695b21d72-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9ed50fe60e4c3cfba5d8d51afc0a1ba695b21d72-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [2]:
# load URDF
mechanism = AtlasSim.addflatground!(AtlasRobot.mechanism())
remove_fixed_tree_joints!(mechanism);

In [3]:
# create optimizer
using MathOptInterface
using OSQP.MathOptInterfaceOSQP
const MOI = MathOptInterface
optimizer = OSQPOptimizer()
MOI.set!(optimizer, OSQPSettings.Verbose(), false)
MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-5)
MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-5)
MOI.set!(optimizer, OSQPSettings.MaxIter(), 5000)
MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior

In [4]:
# create low level controller
lowlevel = MomentumBasedController{4}(mechanism, optimizer);
for body in bodies(mechanism)
    for point in RigidBodyDynamics.contact_points(body)
        position = RigidBodyDynamics.Contact.location(point)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        μ = point.model.friction.μ
        contact = addcontact!(lowlevel, body, position, normal, μ)
        contact.maxnormalforce[] = 1e6 # TODO
        contact.weight[] = 1e-3
    end
end

In [5]:
# create standing controller
info = AtlasSim.atlasrobotinfo(mechanism)
nominalstate = MechanismState(mechanism)
AtlasSim.initialize!(nominalstate, info)
pelvis = successor(info.floatingjoint, mechanism)
comref = center_of_mass(nominalstate) - FreeVector3D(root_frame(mechanism), 0., 0., 0.05)
standingcontroller = StandingController(lowlevel, collect(values(info.feet)), pelvis, nominalstate, comref=comref);

In [6]:
# create LCM control publisher
publisher = LCMControlPublisher(info, standingcontroller;
    robot_state_channel="EST_ROBOT_STATE",
    robot_command_channel="ATLAS_COMMAND");

In [7]:
# run simulation
sol = AtlasSim.run(final_time = 10.0);
flush(stdout)

Publishing START_MIT_STAND


In [8]:
# checks
finalstate = MechanismState(mechanism)
copyto!(finalstate, sol.u[end])
@test sol.retcode == :Success
@test center_of_mass(finalstate) ≈ comref atol=2e-2

Test Passed

Simulated 10.0 s in 14.885309275 s (0.6718033072241961 x realtime).
